In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import pandas as pd

In [24]:
train_df = pd.read_excel('data/好大夫在线_非标准化疾病诉求的简单分诊数据集/train.xlsx', engine='openpyxl')
test_df = pd.read_excel('data/好大夫在线_非标准化疾病诉求的简单分诊数据集/test.xlsx', engine='openpyxl')

In [46]:
len(train_df)

7844

In [25]:
train_df.head(2)

,id,age,diseaseName,conditionDesc,title,hopeHelp,label
0,1,50+,肺部积水等,右腹处疼痛 伴气短 检查出肺部有积水等症状,入院时间咨询,早上好 请问下**今天去医院可以安排吗,8
1,2,30+,怎么才能知道自己缺钙,有时候腰膝酸软四肢无力感觉，睡眠不是太好，总是爱做梦,不知道是缺钙还是肾虚,请问这是肾虚症状还是缺钙症状啊,4


In [52]:
for df in [train_df, test_df]:
    df['text'] = str(df['title']) + str(df['conditionDesc']) + str(df['hopeHelp']) + str(df['diseaseName'])

In [53]:
train_df[train_df['text'].isna()]

,id,age,diseaseName,conditionDesc,title,hopeHelp,label,text


In [56]:
train_df.dropna(subset=["text"])

,id,age,diseaseName,conditionDesc,title,hopeHelp,label,text
0,1,50+,肺部积水等,右腹处疼痛 伴气短 检查出肺部有积水等症状,入院时间咨询,早上好 请问下**今天去医院可以安排吗,8,0 入院时间咨询\n1 ...
1,2,30+,怎么才能知道自己缺钙,有时候腰膝酸软四肢无力感觉，睡眠不是太好，总是爱做梦,不知道是缺钙还是肾虚,请问这是肾虚症状还是缺钙症状啊,4,0 入院时间咨询\n1 ...
2,3,20+,舌头发麻，右下唇僵硬，带有发烧,舌头发麻，右下唇僵硬，带有发烧，嗓子有炎症,舌头发麻，右下唇僵硬，带有发烧,给一些建议，是否需要进行下一步检查,4,0 入院时间咨询\n1 ...
3,4,30+,后背不适，胸口对应处,因经常嗳气，胃吃完顶人，后就医，胃镜显示浅表性胃炎，吃药一周余，有所好转，但后背一直不舒服，...,后背不适，胸口对应处，不是疼，不是酸痛，就是难受,如何调整治疗,4,0 入院时间咨询\n1 ...
4,5,40+,喉咙充血，有异物感，不痛。有少许痰,两个月前，喉部充血，咳嗽，去医院看过，咽炎，现在咳了，就是喉咙充血，有异物感。,看吃点什么药能尽快好起来,看吃点什么药能尽快好起来,0,0 入院时间咨询\n1 ...
...,...,...,...,...,...,...,...,...
7839,7840,30+,桥本氏甲状腺炎,脚软，如晚上睡不好时脖子会出汗,想知道能怀孕不，会不会很严重,想知道能怀孕不，会不会很严重,6,0 入院时间咨询\n1 ...
7840,7841,0+,右下腹疼痛伴有恶心,右下腹痛还有恶心昨天下午4.00开始疼痛,怀疑阑尾炎不确定,看不看挂那个科室,4,0 入院时间咨询\n1 ...
7841,7842,30+,免疫力低下,"女,33岁。麻烦看看结果，这两项异常值代表什么？请大夫给予解答。谢谢",请大夫帮忙看看结果,请大夫帮忙看看结果,4,0 入院时间咨询\n1 ...
7842,7843,10+,背部肝区隐痛，腹涨恶心，没食欲\n腹涨，没食欲\n背部不舒服\n胸口中间闷痛,"男,17岁。干完体力活搬重东西后，出现胸口中间闷痛，呼吸不畅很难受，有慢性鼻炎",胸口中间闷痛,什么引起的,4,0 入院时间咨询\n1 ...


In [76]:
train_df = train_df[0:100]

In [77]:
from sklearn.model_selection import train_test_split
X = list(train_df['text'])
y = list(train_df['label'])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [78]:
len(X_train)

80

In [79]:
from corpus_preprocess.haodaifu_process import HaoDaiFuDataset, Processor
from corpus_preprocess import haodaifu_process
from torch.utils.data import DataLoader

In [80]:
from corpus_preprocess.encoders import BertEncoder
from utils.model_utils import device
import torch
processor = Processor()
encoder = BertEncoder()
def collate_batch(batch):
    # bert encode
    label_list, text_list, token_type_ids_list, attention_mask_list = [], [], [], []
    for x, y in batch:
        label_list.append(y)
        input_ids, token_type_ids, attention_mask = encoder.encode_ids(processor.process(x))
        text_list.append(input_ids)
        token_type_ids_list.append(token_type_ids)
        attention_mask_list.append(attention_mask)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = torch.tensor(text_list)
    type_ids = torch.tensor(token_type_ids_list)
    atten_masks = torch.tensor(attention_mask_list)
    return (text_list.to(device), type_ids.to(device), atten_masks.to(device)), label_list.to(device)

In [81]:
train_ds = HaoDaiFuDataset(X_train, y_train)
valid_ds = HaoDaiFuDataset(X_valid, y_valid)

vocab_size = 100
batch_size = 64
train_dl = DataLoader(train_ds, 
                        batch_size=batch_size,
                        shuffle=True, 
                        collate_fn=collate_batch)
val_dl = DataLoader(valid_ds, 
                    batch_size=batch_size, 
                    collate_fn=collate_batch)

In [82]:
# for x, y in train_dl:
#     print(x)

In [83]:
from models.bert import MyBert
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error

In [84]:
LABELS = list(range(10))
LABELS

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [86]:
class Runner(object):
    def __init__(self, vocab_size, labels):
        super().__init__()
        self.model = MyBert(num_tags=len(labels))

    def run(self, mth='train'):
        if mth == "train":
            self.train_model()

    def train_model(self, epochs=1, lr=0.001):
        parameters = filter(lambda p: p.requires_grad, self.model.parameters())
        optimizer = torch.optim.Adam(parameters, lr=lr)
        for i in range(epochs):
            self.model.train()
            sum_loss = 0.0
            total = 0
            for x, y in train_dl:
                y_pred = self.model(x)
                optimizer.zero_grad()
                print(y_pred.shape, y.shape)
                loss = F.cross_entropy(y_pred, y)
                loss.backward()
                optimizer.step()
                sum_loss += loss.item()*y.shape[0]
                total += y.shape[0]
            # metric on train-data
            train_loss, train_acc, train_rmse = self.validation_metrics(train_dl)
            print("train loss %.3f, train accuracy %.3f, and train rmse %.3f" % (sum_loss/total, train_acc, train_rmse))
            val_loss, val_acc, val_rmse = self.validation_metrics(val_dl)
            # if i % 5 == 1:
            print("val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_acc, val_rmse))

    def validation_metrics(self, valid_dl):
        self.model.eval()
        correct = 0
        total = 0
        sum_loss = 0.0
        sum_rmse = 0.0
        for x, y in valid_dl:
            # x = x.long()
            # y = y.long()
            y_hat = self.model(x)
            loss = F.cross_entropy(y_hat, y)
            pred = torch.max(y_hat, 1)[1]
            correct += (pred == y).float().sum()
            total += y.shape[0]
            sum_loss += loss.item()*y.shape[0]
            sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
        return sum_loss/total, correct/total, sum_rmse/total

runner = Runner(vocab_size=vocab_size, labels=LABELS)
runner.run(mth="train")